<a href="https://colab.research.google.com/github/ayaanzhaque/SuiSense/blob/master/notebooks/BERT/workingBertModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
#importing relevant libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB

import torch

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

## Importing the dataset
We'll use pandas to read the dataset and load it into a dataframe.

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/ayaanzhaque/SuiSense/master/data/scheme1.csv')
og_batch_1 = df[['selftext', 'is_suicide']]



For performance reasons, we'll only use 2,000 sentences from the dataset

In [ ]:
#batch_1_start = og_batch_1.head(60)
#batch_1_end = og_batch_1.tail(60)
#test_batch_1 = pd.concat([batch_1_start, batch_1_end], ignore_index=True)
batch_1 = og_batch_1.rename(columns={'selftext': 0, 'is_suicide': 1})
batch_1.head()

,0,1
0,We understand that most people who reply immed...,0
1,Welcome to /r/depression's check-in post - a p...,0
2,I've been feeling really depressed and lonely ...,0
3,I literally broke down crying and asked to go ...,0
4,Any kind soul want to give a depressed person ...,0


We can ask pandas how many sentences are labeled as "positive" (value 1) and how many are labeled "negative" (having the value 0)

In [ ]:
batch_1[1].value_counts()

1    980
0    917
Name: 1, dtype: int64

## Loading the Pre-trained BERT model
Let's now load a pre-trained BERT model. 

In [ ]:
# For DistilBERT:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Right now, the variable `model` holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

## Model #1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to so some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [ ]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=128)))

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

Our dataset is now in the `padded` variable, we can view its dimensions below:

In [ ]:
np.array(padded).shape

(1897, 128)

### Masking
If we directly send `padded` to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1897, 128)

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
print(features)

[[ 0.15627109 -0.20722139 -0.03962984 ...  0.15881492  0.36695075
   0.46319005]
 [-0.05789456 -0.01733888 -0.10708018 ...  0.01880779 -0.06970225
   0.54524034]
 [ 0.19605026  0.20534156  0.24803329 ...  0.15781963  0.45455933
  -0.41783777]
 ...
 [ 0.04628711  0.53464365 -0.19218408 ... -0.34011787  0.7708874
   0.6021303 ]
 [ 0.24700873  0.2810834   0.21017519 ... -0.1912898   0.2667738
  -0.13201511]
 [ 0.16641247  0.16862674  0.03572434 ... -0.15101409  0.5139446
   0.24030764]]


The labels indicating which sentence is positive and negative now go into the `labels` variable

In [ ]:
labels = batch_1[1]
labels.head()

0    0
1    0
2    0
3    0
4    0
Name: 1, dtype: int64

## Model #2: Train/Test Split
Let's now split our datset into a training set and testing set (even though we're using 2,000 sentences from the SST2 training set).

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.25, random_state=42, stratify=labels)

In [ ]:
# parameters = {'C': np.linspace(0.0001, 100, 20)}
# grid_search = GridSearchCV(LogisticRegression(), parameters)
# grid_search.fit(train_features, train_labels)

# print('best parameters: ', grid_search.best_params_)
# print('best scrores: ', grid_search.best_score_)

We now train the LogisticRegression model. If you've chosen to do the gridsearch, you can plug the value of C into the model declaration (e.g. `LogisticRegression(C=5.2)`).

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import ModelCheckpoint

model3 = Sequential()
model3.add(Dense(1, activation='sigmoid'))

model3.add(Dense(10, activation='relu'))
# model3.add(Dense(1, activation='sigmoid'))
model3.add(Dense(2, activation = 'softmax')) # this gives the probability distribution for each class, should be 1 hot encoded, maybe we take this out of the model and only use it when predicting

model3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


checkpoint = ModelCheckpoint('/content/bertmodelRegLabels.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  


history = model3.fit(train_features, train_labels, batch_size=32, epochs=100, verbose=1, callbacks=[checkpoint], validation_data=(test_features, test_labels))

Epoch 1/100
32/45 [====================>.........] - ETA: 0s - loss: 0.6988 - accuracy: 0.4824
Epoch 00001: val_loss improved from inf to 0.69109, saving model to /content/bertmodelRegLabels.h5
45/45 [==============================] - 0s 7ms/step - loss: 0.6973 - accuracy: 0.4887 - val_loss: 0.6911 - val_accuracy: 0.5937
Epoch 2/100
42/45 [===========================>..] - ETA: 0s - loss: 0.6905 - accuracy: 0.5885
Epoch 00002: val_loss improved from 0.69109 to 0.68735, saving model to /content/bertmodelRegLabels.h5
45/45 [==============================] - 0s 3ms/step - loss: 0.6899 - accuracy: 0.5942 - val_loss: 0.6873 - val_accuracy: 0.5705
Epoch 3/100
42/45 [===========================>..] - ETA: 0s - loss: 0.6856 - accuracy: 0.5580
Epoch 00003: val_loss improved from 0.68735 to 0.68429, saving model to /content/bertmodelRegLabels.h5
45/45 [==============================] - 0s 3ms/step - loss: 0.6852 - accuracy: 0.5654 - val_loss: 0.6843 - val_accuracy: 0.6084
Epoch 4/100
39/45 [====

In [ ]:
from keras.models import load_model

loadedModel = load_model('/content/bertmodelRegLabels.h5')
loadedModel.evaluate(test_features, test_labels)

15/15 [==============================] - 0s 1ms/step - loss: 0.6019 - accuracy: 0.6989


[0.6018891930580139, 0.6989473700523376]

<img src="https://jalammar.github.io/images/distilBERT/bert-training-logistic-regression.png" />

## Evaluating Model #2
So how well does our model do in classifying sentences? One way is to check the accuracy against the testing dataset:

How good is this score? What can we compare it against? Let's first look at a dummy classifier:

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.509 (+/- 0.06)


So our model clearly does better than a dummy classifier. But how does it compare against the best models?

## Proper SST2 scores
For reference, the [highest accuracy score](http://nlpprogress.com/english/sentiment_analysis.html) for this dataset is currently **96.8**. DistilBERT can be trained to improve its score on this task – a process called **fine-tuning** which updates BERT’s weights to make it achieve a better performance in this sentence classification task (which we can call the downstream task). The fine-tuned DistilBERT turns out to achieve an accuracy score of **90.7**. The full size BERT model achieves **94.9**.



And that’s it! That’s a good first contact with BERT. The next step would be to head over to the documentation and try your hand at [fine-tuning](https://huggingface.co/transformers/examples.html#glue). You can also go back and switch from distilBERT to BERT and see how that works.